In [120]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV

# Lending Club

## 1.  

In [127]:
df_lend = pd.read_excel("/Users/youssouf/Downloads/Lending_Club.xlsx")
df_lend.head()

,Unnamed: 0,id,member_id,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,...,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,0,263591,545710,20000.0,60 months,17.93,342.94,E,E5,Wylie ISD,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
1,1,1613916,69664096,30000.0,36 months,11.99,996.29,C,C1,Sergeant,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
2,2,818934,8965180,21500.0,36 months,11.99,714.01,B,B3,Designer,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
3,3,1606612,70572960,10000.0,36 months,13.67,340.18,C,C3,NaN,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
4,4,1639932,68589517,5000.0,36 months,8.49,157.82,B,B1,Sr. Manufacturing Engineer,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN


In [128]:
df_lend.shape

(25000, 135)

In [129]:
a = "rrr"

In [130]:
a.startswith

<function str.startswith>

In [131]:
for i in df_lend.columns :
    if i.startswith("total") :
        print(i)

total_acc
total_bal_il
total_rev_hi_lim
total_cu_tl
total_bal_ex_mort
total_bc_limit
total_il_high_credit_limit


In [132]:
df_lend = df_lend.drop(["Unnamed: 0"], axis = 1)
df_lend.head()

,id,member_id,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,263591,545710,20000.0,60 months,17.93,342.94,E,E5,Wylie ISD,1 year,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
1,1613916,69664096,30000.0,36 months,11.99,996.29,C,C1,Sergeant,10+ years,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
2,818934,8965180,21500.0,36 months,11.99,714.01,B,B3,Designer,1 year,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
3,1606612,70572960,10000.0,36 months,13.67,340.18,C,C3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
4,1639932,68589517,5000.0,36 months,8.49,157.82,B,B1,Sr. Manufacturing Engineer,10+ years,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN


In [133]:
df_lend["loan_status"].value_counts()

Current                                                12063
Fully Paid                                              9733
Charged Off                                             2557
Late (31-120 days)                                       336
In Grace Period                                          174
Late (16-30 days)                                        101
Does not meet the credit policy. Status:Fully Paid        28
Does not meet the credit policy. Status:Charged Off        5
Default                                                    2
Name: loan_status, dtype: int64

In [134]:
df_lend = df_lend[(df_lend["loan_status"] ==  "Fully Paid") | (df_lend["loan_status"] ==  "Charged Off")]
df_lend.head()


,id,member_id,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,263591,545710,20000.0,60 months,17.93,342.94,E,E5,Wylie ISD,1 year,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
2,818934,8965180,21500.0,36 months,11.99,714.01,B,B3,Designer,1 year,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
3,1606612,70572960,10000.0,36 months,13.67,340.18,C,C3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
5,756884,107370446,16150.0,60 months,30.74,529.88,F,F5,Meat and produce unloader,10+ years,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
8,1503361,91060858,35000.0,36 months,12.79,1175.76,C,C1,Business Manaer,9 years,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN


In [135]:
# total_cur_bal is not among the variables
df_lend = df_lend[["home_ownership","annual_inc","dti","fico_range_low","revol_bal","total_bal_il","loan_status"]]
df_lend.head()

,home_ownership,annual_inc,dti,fico_range_low,revol_bal,total_bal_il,loan_status
0,MORTGAGE,44304.0,18.47,690.0,5673.0,NaN,Charged Off
2,RENT,50000.0,29.62,735.0,14071.0,NaN,Fully Paid
3,RENT,64400.0,16.68,675.0,8057.0,8767.0,Fully Paid
5,RENT,38500.0,33.73,660.0,12666.0,34191.0,Charged Off
8,MORTGAGE,118000.0,26.66,665.0,105605.0,11146.0,Fully Paid


In [136]:
df_lend.shape

(12290, 7)

In [137]:
### Count Na values
df_lend.isna().mean()

home_ownership    0.000000
annual_inc        0.000000
dti               0.000081
fico_range_low    0.000000
revol_bal         0.000000
total_bal_il      0.776322
loan_status       0.000000
dtype: float64

In [138]:
### Delete missing values
df_lend = df_lend.dropna()

In [139]:
df_lend.isna().mean()

home_ownership    0.0
annual_inc        0.0
dti               0.0
fico_range_low    0.0
revol_bal         0.0
total_bal_il      0.0
loan_status       0.0
dtype: float64

In [140]:
df_lend.shape

(2748, 7)

In [141]:
def discrete_home_ownership(x) :
    if (x == "RENT") :
        return 0
    elif (x == "MORTGAGE") :
        return 1
    else :
        return(2)

In [142]:
df_lend["loan_status"] = df_lend["loan_status"].apply(lambda x : 0 if (x == "Charged Off") else 1)
df_lend["home_ownership"] = df_lend["home_ownership"].apply(lambda x : discrete_home_ownership(x))
df_lend.head()

,home_ownership,annual_inc,dti,fico_range_low,revol_bal,total_bal_il,loan_status
3,0,64400.0,16.68,675.0,8057.0,8767.0,1
5,0,38500.0,33.73,660.0,12666.0,34191.0,0
8,1,118000.0,26.66,665.0,105605.0,11146.0,1
15,1,54000.0,19.00,660.0,11988.0,20528.0,0
30,0,48000.0,5.69,665.0,8347.0,0.0,1


In [143]:
### Training, Validation, Testing
train, validate, test = \
              np.split(df_lend.sample(frac=1, random_state=42), 
                       [int(.6*len(df_lend)), int(.8*len(df_lend))])

In [144]:
train.head()


,home_ownership,annual_inc,dti,fico_range_low,revol_bal,total_bal_il,loan_status
22863,1,69000.0,22.73,690.0,4905.0,32741.0,0
23967,1,42000.0,28.40,675.0,7615.0,33367.0,1
11705,1,53000.0,29.89,720.0,9040.0,25247.0,0
14745,2,130000.0,32.66,675.0,15365.0,79123.0,1
24152,0,45000.0,13.49,685.0,7992.0,26964.0,0


In [145]:
validate.head()


,home_ownership,annual_inc,dti,fico_range_low,revol_bal,total_bal_il,loan_status
9255,1,70000.0,35.37,660.0,44095.0,46539.0,1
20164,2,87000.0,14.30,705.0,25428.0,14990.0,0
1314,1,45000.0,12.83,725.0,18312.0,1760.0,1
17809,0,52000.0,22.85,665.0,5860.0,10823.0,1
23037,1,125000.0,40.71,690.0,120235.0,80806.0,1


In [146]:
test.head()


,home_ownership,annual_inc,dti,fico_range_low,revol_bal,total_bal_il,loan_status
7928,0,35000.0,8.47,670.0,4300.0,25659.0,1
7057,0,106000.0,7.26,670.0,7210.0,22285.0,0
13132,1,120000.0,25.99,705.0,12446.0,177246.0,0
10396,2,70000.0,29.30,670.0,4737.0,30238.0,0
4304,1,65802.0,13.79,760.0,7332.0,3691.0,1


## 2. Models : Linear Discriminant Analysis, Logistic Regression, Decision Tree 

### Linear Discriminant Analysis

In [147]:
# define dataset
X, y = train[["home_ownership","annual_inc","dti","fico_range_low","revol_bal","total_bal_il"]], train[["loan_status"]]
# define model
model = LinearDiscriminantAnalysis()
# define model evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid
grid = dict()
grid['solver'] = ['svd', 'lsqr', 'eigen']
# define search
search = GridSearchCV(model, grid, scoring='accuracy', cv=cv, n_jobs=-1)
# perform the search
results = search.fit(X, y)
# summarize
print('Mean Accuracy: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)

Mean Accuracy: 0.780
Config: {'solver': 'svd'}


/Users/youssouf/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [148]:
model.fit(X, y)
#yhat = model.predict([row])

/Users/youssouf/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LinearDiscriminantAnalysis()

### Logistic Regression

In [149]:
# import the class
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression()

X_train,y_train = train[["home_ownership","annual_inc","dti","fico_range_low","revol_bal","total_bal_il"]], train[["loan_status"]]


# fit the model with data
logreg.fit(X_train,y_train)

X_test = train[["home_ownership","annual_inc","dti","fico_range_low","revol_bal","total_bal_il"]]

#
y_pred=logreg.predict(X_test)

/Users/youssouf/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [150]:
y_train.value_counts()

loan_status
1              1284
0               364
dtype: int64

In [4]:
def field(schema) :
    a = schema.split("|-- ")
    keys_list = [i.split(": ")[0] for i in a[1:]]
    values_list = [((i.split(": ")[1]).split("(")[0]).strip() for i in a[1:] ]
    a_dictionary = dict(zip(keys_list, values_list))
    return(keys_list)

In [5]:
s = """ |-- msisdn_in: string (nullable = true)
 |-- nbre_msisdn_in: long (nullable = true)
 |-- nd_gaia: string (nullable = true)
 |-- nbre_msisdn_gaia: long (nullable = true)
 |-- nd_ocb: string (nullable = true)
 |-- nbre_msisdn_ocb: long (nullable = true)
 |-- offre_in: string (nullable = true)
 |-- prepaid_in: string (nullable = true)
 |-- statut_in: string (nullable = true)
 |-- created_date_in: string (nullable = true)
 |-- active_date_in: string (nullable = true)
 |-- msisdn_in_formatd: string (nullable = true)
 |-- dv_gaia: string (nullable = true)
 |-- nom_prenoms_gaia: string (nullable = true)
 |-- qualite_client_gaia: string (nullable = true)
 |-- type_client_gaia: string (nullable = true)
 |-- categorie_gaia: string (nullable = true)
 |-- portefeuille_gaia: string (nullable = true)
 |-- ncli_gaia: string (nullable = true)
 |-- siren_gaia: string (nullable = true)
 |-- ne_gaia: string (nullable = true)
 |-- type_dossier_gaia: string (nullable = true)
 |-- ndos_gaia: string (nullable = true)
 |-- contact_mobile_gaia: string (nullable = true)
 |-- second_mobile_gaia: string (nullable = true)
 |-- e_mail_gaia: string (nullable = true)
 |-- adr_postale_gaia: string (nullable = true)
 |-- produit_gaia: string (nullable = true)
 |-- type_abo_gaia: string (nullable = true)
 |-- code_offre_gaia: string (nullable = true)
 |-- libelle_service_gaia: string (nullable = true)
 |-- type_msan_gaia: string (nullable = true)
 |-- date_sce_gaia: string (nullable = true)
 |-- etat_ligne_gaia: string (nullable = true)
 |-- raison_susp_gaia: string (nullable = true)
 |-- date_etat_gaia: string (nullable = true)
 |-- adress_install_gaia: string (nullable = true)
 |-- repartiteur_gaia: string (nullable = true)
 |-- sr_gaia: string (nullable = true)
 |-- autocom_gaia: string (nullable = true)
 |-- groupe_fact_gaia: string (nullable = true)
 |-- etat_client_gaia: string (nullable = true)
 |-- date_ms_gaia: string (nullable = true)
 |-- commune_gaia: string (nullable = true)
 |-- voie_gaia: string (nullable = true)
 |-- msisdn_gaia_formatd: string (nullable = true)
 |-- nbre_ne_gaia: long (nullable = true)
 |-- cat_ocb: string (nullable = true)
 |-- mar_ocb: string (nullable = true)
 |-- ndg_ocb: string (nullable = true)
 |-- ne_ocb: string (nullable = true)
 |-- nsg_ocb: string (nullable = true)
 |-- tax_ocb: string (nullable = true)
 |-- ty_ocb: string (nullable = true)
 |-- site_ocb: string (nullable = true)
 |-- date_insert_ocb: string (nullable = true)
 |-- msisdn_ocb_formatd: string (nullable = true)
 |-- nbre_ne_ocb: long (nullable = true)
 |-- msisdn: string (nullable = true)
 |-- date_insert: string (nullable = true)
 |-- type_ligne: string (nullable = false)"""

In [30]:
d = field(s)

In [36]:
#["\"" + literal_eval(str(i)) + "\"" for i in d]

In [25]:
from ast import *

In [35]:
#[literal_eval(i) for i in d]

In [56]:
d.index("nd_gaia")

2

In [54]:
[ print(("\"" + i + "\"").replace("'","") + ",") for i in d]

"msisdn_in",
"nbre_msisdn_in",
"nd_gaia",
"nbre_msisdn_gaia",
"nd_ocb",
"nbre_msisdn_ocb",
"offre_in",
"prepaid_in",
"statut_in",
"created_date_in",
"active_date_in",
"msisdn_in_formatd",
"dv_gaia",
"nom_prenoms_gaia",
"qualite_client_gaia",
"type_client_gaia",
"categorie_gaia",
"portefeuille_gaia",
"ncli_gaia",
"siren_gaia",
"ne_gaia",
"type_dossier_gaia",
"ndos_gaia",
"contact_mobile_gaia",
"second_mobile_gaia",
"e_mail_gaia",
"adr_postale_gaia",
"produit_gaia",
"type_abo_gaia",
"code_offre_gaia",
"libelle_service_gaia",
"type_msan_gaia",
"date_sce_gaia",
"etat_ligne_gaia",
"raison_susp_gaia",
"date_etat_gaia",
"adress_install_gaia",
"repartiteur_gaia",
"sr_gaia",
"autocom_gaia",
"groupe_fact_gaia",
"etat_client_gaia",
"date_ms_gaia",
"commune_gaia",
"voie_gaia",
"msisdn_gaia_formatd",
"nbre_ne_gaia",
"cat_ocb",
"mar_ocb",
"ndg_ocb",
"ne_ocb",
"nsg_ocb",
"tax_ocb",
"ty_ocb",
"site_ocb",
"date_insert_ocb",
"msisdn_ocb_formatd",
"nbre_ne_ocb",
"msisdn",
"date_insert",
"type_ligne",


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [ ]:
#.replace("'","").strip("'")

In [49]:
print('"msisdn"')

"msisdn"


In [1]:
n = """2720321561

2722440005

2722443197

2722434154

2722439847

2721280606

2721351893

2721359110

2721252841

2730682448"""

In [2]:
n

'2720321561\n\n2722440005\n\n2722443197\n\n2722434154\n\n2722439847\n\n2721280606\n\n2721351893\n\n2721359110\n\n2721252841\n\n2730682448'

In [5]:
[ print(("\"" + i + "\"").replace("'","") + ",") for i in n.split("\n\n")]   

"2720321561",
"2722440005",
"2722443197",
"2722434154",
"2722439847",
"2721280606",
"2721351893",
"2721359110",
"2721252841",
"2730682448",


[None, None, None, None, None, None, None, None, None, None]

In [1]:
import argparse

In [2]:
ap = argparse.ArgumentParser()

In [ ]:
ap.add_argument()